In [3]:
import catboost as catb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd 
import numpy as np


In [68]:
data = pd.read_csv('train.csv')
data.columns

Index(['id', 'age', 'alcohol_consumption_per_week',
       'physical_activity_minutes_per_week', 'diet_score',
       'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi',
       'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'education_level',
       'income_level', 'smoking_status', 'employment_status',
       'family_history_diabetes', 'hypertension_history',
       'cardiovascular_history', 'diagnosed_diabetes'],
      dtype='object')

In [69]:
test = pd.read_csv('test.csv')

In [70]:
data.head()
data=data.set_index('id')

test = test.set_index('id')

In [71]:
data['gender'] = data['gender'].astype('category')
data['ethnicity'] = data['ethnicity'].astype('category')
data['education_level'] = data['education_level'].astype('category')
data['income_level'] = data['income_level'].astype('category')
data['smoking_status'] = data['smoking_status'].astype('category')
data['employment_status'] = data['employment_status'].astype('category')



In [72]:
X = data.drop(columns='diagnosed_diabetes')
y = data['diagnosed_diabetes'].copy()
y.sum()/len(y)

0.6232957142857143

In [73]:
X_train ,X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state=42 , stratify=y)
y_train.sum()/len(y_train),y_test.sum()/len(y_test)


(0.6232964285714285, 0.6232928571428571)

In [11]:
from sklearn.metrics import roc_auc_score

In [74]:
cat_features = [
    "gender", "ethnicity", "education_level",
    "income_level", "smoking_status", "employment_status"
]

In [48]:
from catboost import CatBoostClassifier , Pool

In [75]:
test
test['gender'] = test['gender'].astype('category')
test['ethnicity'] = test['ethnicity'].astype('category')
test['education_level'] = test['education_level'].astype('category')
test['income_level'] = test['income_level'].astype('category')
test['smoking_status'] = test['smoking_status'].astype('category')
test['employment_status'] = test['employment_status'].astype('category')

In [14]:
param_grid = {
    "depth": [4, 5, 6, 7, 8, 9, 10],
    "learning_rate": [0.005, 0.01, 0.02, 0.03],
    "l2_leaf_reg": [1, 3, 5, 7, 10, 15, 20],
    "bagging_temperature": [0.1, 0.5, 1.0, 2.0, 5.0],
    "random_strength": [0.5, 1, 1.5, 2, 3],
    "border_count": [32, 64, 128, 254],
    "grow_policy": ["SymmetricTree", "Lossguide"]
}


In [28]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

model = CatBoostClassifier(
    iterations=3000,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=0,
    task_type="GPU",       
    devices='0'
)

In [16]:
search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    scoring="roc_auc",
    cv=3,
    n_iter=20,
    verbose=3,
    random_state=42,
    n_jobs=1
)

In [60]:
train_pool = Pool(X_train , y_train , cat_features = cat_features)
valid_pool = Pool(X_test , y_test , cat_features = cat_features)
all_train_Pool=Pool(X,y,cat_features = cat_features)

In [18]:
search.fit(
    X_train,
    y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    verbose=100,
    early_stopping_rounds=200
)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6681513	best: 0.6681513 (0)	total: 122ms	remaining: 6m 7s
100:	test: 0.6971585	best: 0.6971585 (100)	total: 9.64s	remaining: 4m 36s
200:	test: 0.7001724	best: 0.7001724 (200)	total: 18.9s	remaining: 4m 22s
300:	test: 0.7016011	best: 0.7016011 (300)	total: 28.3s	remaining: 4m 14s
400:	test: 0.7029906	best: 0.7029906 (400)	total: 37s	remaining: 3m 59s
500:	test: 0.7040625	best: 0.7040625 (500)	total: 46.7s	remaining: 3m 52s
600:	test: 0.7051034	best: 0.7051058 (599)	total: 55.1s	remaining: 3m 39s
700:	test: 0.7057384	best: 0.7057384 (700)	total: 1m 4s	remaining: 3m 30s
800:	test: 0.7062041	best: 0.7062083 (798)	total: 1m 12s	remaining: 3m 19s
900:	test: 0.7065282	best: 0.7065306 (893)	total: 1m 21s	remaining: 3m 9s
1000:	test: 0.7068465	best: 0.7068481 (998)	total: 1m 29s	remaining: 2m 59s
1100:	test: 0.7069595	best: 0.7069595 (1100)	total: 1m 37s	remaining: 2m 48s
1200:	test: 0.7071266	best: 0.7071266 (1200)	total: 1m 45s	remaining: 2m 38s
1300:	test: 0.7072921	best: 0.707292

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6669752	best: 0.6669752 (0)	total: 80.3ms	remaining: 4m
100:	test: 0.6970644	best: 0.6970644 (100)	total: 8.27s	remaining: 3m 57s
200:	test: 0.7007098	best: 0.7007098 (200)	total: 16.9s	remaining: 3m 56s
300:	test: 0.7023274	best: 0.7023274 (300)	total: 25.8s	remaining: 3m 51s
400:	test: 0.7039740	best: 0.7039740 (400)	total: 34.9s	remaining: 3m 46s
500:	test: 0.7052974	best: 0.7052974 (500)	total: 43.1s	remaining: 3m 34s
600:	test: 0.7061931	best: 0.7061931 (600)	total: 52.1s	remaining: 3m 27s
700:	test: 0.7067199	best: 0.7067199 (700)	total: 1m 1s	remaining: 3m 20s
800:	test: 0.7071131	best: 0.7071131 (800)	total: 1m 9s	remaining: 3m 10s
900:	test: 0.7073976	best: 0.7073976 (900)	total: 1m 17s	remaining: 3m 1s
1000:	test: 0.7076557	best: 0.7076557 (1000)	total: 1m 26s	remaining: 2m 53s
1100:	test: 0.7077619	best: 0.7077619 (1100)	total: 1m 35s	remaining: 2m 45s
1200:	test: 0.7078783	best: 0.7078796 (1199)	total: 1m 45s	remaining: 2m 37s
1300:	test: 0.7079765	best: 0.707976

...

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6693898	best: 0.6693898 (0)	total: 123ms	remaining: 6m 9s
100:	test: 0.6860886	best: 0.6860886 (100)	total: 13.9s	remaining: 6m 40s
200:	test: 0.6897766	best: 0.6897766 (200)	total: 26.5s	remaining: 6m 8s
300:	test: 0.6932200	best: 0.6932200 (300)	total: 38.8s	remaining: 5m 48s
400:	test: 0.6952972	best: 0.6952972 (400)	total: 51.4s	remaining: 5m 33s
500:	test: 0.6970676	best: 0.6970676 (500)	total: 1m 3s	remaining: 5m 17s
600:	test: 0.6983403	best: 0.6983403 (600)	total: 1m 15s	remaining: 5m 3s
700:	test: 0.6994791	best: 0.6994791 (700)	total: 1m 28s	remaining: 4m 49s
800:	test: 0.7005151	best: 0.7005151 (800)	total: 1m 40s	remaining: 4m 35s
900:	test: 0.7013464	best: 0.7013464 (900)	total: 1m 52s	remaining: 4m 21s
1000:	test: 0.7020539	best: 0.7020539 (1000)	total: 2m 4s	remaining: 4m 8s
1100:	test: 0.7027283	best: 0.7027283 (1100)	total: 2m 17s	remaining: 3m 56s
1200:	test: 0.7032197	best: 0.7032197 (1200)	total: 2m 30s	remaining: 3m 45s
1300:	test: 0.7037205	best: 0.7037

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6823601	best: 0.6823601 (0)	total: 97.2ms	remaining: 4m 51s
100:	test: 0.7042745	best: 0.7042745 (100)	total: 7.89s	remaining: 3m 46s
200:	test: 0.7105146	best: 0.7105146 (200)	total: 15.4s	remaining: 3m 34s
300:	test: 0.7154298	best: 0.7154298 (300)	total: 23.1s	remaining: 3m 27s
400:	test: 0.7186009	best: 0.7186009 (400)	total: 30.7s	remaining: 3m 18s
500:	test: 0.7205571	best: 0.7205571 (500)	total: 38.4s	remaining: 3m 11s
600:	test: 0.7220242	best: 0.7220242 (600)	total: 46s	remaining: 3m 3s
700:	test: 0.7231982	best: 0.7231982 (700)	total: 53.6s	remaining: 2m 55s
800:	test: 0.7239904	best: 0.7239904 (800)	total: 1m 1s	remaining: 2m 48s
900:	test: 0.7245632	best: 0.7245632 (900)	total: 1m 9s	remaining: 2m 41s
1000:	test: 0.7249805	best: 0.7249805 (1000)	total: 1m 16s	remaining: 2m 33s
1100:	test: 0.7253205	best: 0.7253205 (1100)	total: 1m 24s	remaining: 2m 26s
1200:	test: 0.7257974	best: 0.7258011 (1198)	total: 1m 32s	remaining: 2m 18s
1300:	test: 0.7261102	best: 0.72611

,estimator,<catboost.cor...001C64E9475E0>
,param_distributions,"{'bagging_temperature': [0.1, 0.5, ...], 'border_count': [32, 64, ...], 'depth': [4, 5, ...], 'grow_policy': ['SymmetricTree', 'Lossguide'], ...}"
,n_iter,20
,scoring,'roc_auc'
,n_jobs,1
,refit,True
,cv,3
,verbose,3
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [21]:
search.best_params_

{'random_strength': 1,
 'learning_rate': 0.03,
 'l2_leaf_reg': 10,
 'grow_policy': 'Lossguide',
 'depth': 5,
 'border_count': 254,
 'bagging_temperature': 0.1}

In [20]:
print(model.get_param("task_type"))


GPU


In [23]:
from sklearn.metrics import roc_auc_score

In [28]:
proba = search.best_estimator_.predict_proba(X_test)[:,1]
roc_auc_score(y_test , proba)

0.7273094943619679

In [41]:
model = CatBoostClassifier(
    iterations=30000,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=0,
    task_type="GPU",       
    devices='0',bagging_temperature=0.1, border_count=254, depth=5, l2_leaf_reg=15, learning_rate=0.01, random_strength=0.5
    
)

In [42]:
model.fit(train_pool , eval_set=valid_pool , verbose=200, use_best_model=True,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6763576	best: 0.6763576 (0)	total: 114ms	remaining: 57m 11s
200:	test: 0.6972961	best: 0.6972961 (200)	total: 21.2s	remaining: 52m 22s
400:	test: 0.7025459	best: 0.7025459 (400)	total: 41.8s	remaining: 51m 23s
600:	test: 0.7053553	best: 0.7053553 (600)	total: 1m 2s	remaining: 51m 7s
800:	test: 0.7080837	best: 0.7080837 (800)	total: 1m 23s	remaining: 50m 56s
1000:	test: 0.7106493	best: 0.7106493 (1000)	total: 1m 44s	remaining: 50m 36s
1200:	test: 0.7125436	best: 0.7125436 (1200)	total: 2m 5s	remaining: 50m 14s
1400:	test: 0.7143233	best: 0.7143233 (1400)	total: 2m 26s	remaining: 49m 44s
1600:	test: 0.7161245	best: 0.7161245 (1600)	total: 2m 48s	remaining: 49m 53s
1800:	test: 0.7173123	best: 0.7173123 (1800)	total: 3m 11s	remaining: 49m 59s
2000:	test: 0.7182951	best: 0.7182951 (2000)	total: 3m 33s	remaining: 49m 41s
2200:	test: 0.7191914	best: 0.7191914 (2200)	total: 3m 55s	remaining: 49m 35s
2400:	test: 0.7198974	best: 0.7198974 (2400)	total: 4m 17s	remaining: 49m 21s
2600:	

In [47]:
model = CatBoostClassifier(
    iterations=10000,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=0,
    task_type="GPU",       
    devices='0',bagging_temperature=1.0, border_count=254, depth=5, learning_rate=0.005, l2_leaf_reg=20, random_strength=1.5
)
model.fit(train_pool , eval_set=valid_pool , verbose=200, use_best_model=True,plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6756073	best: 0.6756073 (0)	total: 115ms	remaining: 19m 13s
200:	test: 0.6925365	best: 0.6925365 (200)	total: 23.9s	remaining: 19m 25s
400:	test: 0.6972469	best: 0.6972469 (400)	total: 47.9s	remaining: 19m 6s
600:	test: 0.7004102	best: 0.7004102 (600)	total: 1m 11s	remaining: 18m 45s
800:	test: 0.7026114	best: 0.7026114 (800)	total: 1m 35s	remaining: 18m 17s
1000:	test: 0.7041551	best: 0.7041551 (1000)	total: 1m 59s	remaining: 17m 55s
1200:	test: 0.7053762	best: 0.7053762 (1200)	total: 2m 21s	remaining: 17m 18s
1400:	test: 0.7064421	best: 0.7064421 (1400)	total: 2m 43s	remaining: 16m 42s
1600:	test: 0.7077295	best: 0.7077295 (1600)	total: 3m 4s	remaining: 16m 9s
1800:	test: 0.7089126	best: 0.7089126 (1800)	total: 3m 26s	remaining: 15m 41s
2000:	test: 0.7100069	best: 0.7100069 (2000)	total: 3m 48s	remaining: 15m 14s
2200:	test: 0.7109070	best: 0.7109070 (2200)	total: 4m 10s	remaining: 14m 47s
2400:	test: 0.7118371	best: 0.7118371 (2400)	total: 4m 32s	remaining: 14m 21s
2600:	

In [30]:
model = CatBoostClassifier(
    iterations=30000,
    loss_function="Logloss",
    eval_metric="AUC",
    verbose=0,
    task_type="GPU",       
    devices='0',
    bagging_temperature = 1.0,
    border_count        = 128,
    depth               = 4,
    grow_policy         = "Lossguide",
    l2_leaf_reg         = 3,
    learning_rate       = 0.01,
    random_strength     = 3,

)
model.fit(train_pool , eval_set=valid_pool , verbose=200, use_best_model=True,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6732238	best: 0.6732238 (0)	total: 59.9ms	remaining: 29m 56s
200:	test: 0.6962301	best: 0.6962301 (200)	total: 7.15s	remaining: 17m 40s
400:	test: 0.7031881	best: 0.7031881 (400)	total: 13.5s	remaining: 16m 38s
600:	test: 0.7065380	best: 0.7065380 (600)	total: 20.2s	remaining: 16m 26s
800:	test: 0.7091551	best: 0.7091551 (800)	total: 27.5s	remaining: 16m 44s
1000:	test: 0.7120470	best: 0.7120470 (1000)	total: 33.8s	remaining: 16m 19s
1200:	test: 0.7142264	best: 0.7142264 (1200)	total: 40s	remaining: 15m 58s
1400:	test: 0.7160293	best: 0.7160293 (1400)	total: 46.4s	remaining: 15m 47s
1600:	test: 0.7176090	best: 0.7176090 (1600)	total: 53s	remaining: 15m 39s
1800:	test: 0.7186747	best: 0.7186747 (1800)	total: 58.8s	remaining: 15m 20s
2000:	test: 0.7194723	best: 0.7194723 (2000)	total: 1m 4s	remaining: 15m 4s
2200:	test: 0.7202098	best: 0.7202098 (2200)	total: 1m 10s	remaining: 14m 50s
2400:	test: 0.7209129	best: 0.7209129 (2400)	total: 1m 15s	remaining: 14m 33s
2600:	test: 0.7

In [48]:


model = CatBoostClassifier(
    iterations=7000,             
    learning_rate=0.02,        
    depth=8,                     
    l2_leaf_reg=3,               
    loss_function="Logloss",   
    eval_metric="AUC",          
    bootstrap_type='Bayesian',   
    random_strength=1.5,        
    bagging_temperature=1.0,     
    random_seed=42,
    od_type="Iter",              
    od_wait=100,                 
    task_type="GPU",       
    devices='0',             
)
model.fit(all_train_Pool , eval_set=valid_pool , verbose=200, use_best_model=True)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6860027	best: 0.6860027 (0)	total: 114ms	remaining: 13m 18s
200:	test: 0.7088269	best: 0.7088269 (200)	total: 21s	remaining: 11m 50s
400:	test: 0.7153913	best: 0.7153913 (400)	total: 42.5s	remaining: 11m 39s
600:	test: 0.7200940	best: 0.7200940 (600)	total: 1m 2s	remaining: 11m 6s
800:	test: 0.7239128	best: 0.7239128 (800)	total: 1m 25s	remaining: 10m 59s
1000:	test: 0.7265170	best: 0.7265170 (1000)	total: 1m 45s	remaining: 10m 33s
1200:	test: 0.7286996	best: 0.7286996 (1200)	total: 2m 7s	remaining: 10m 13s
1400:	test: 0.7306528	best: 0.7306528 (1400)	total: 2m 29s	remaining: 9m 57s
1600:	test: 0.7324609	best: 0.7324609 (1600)	total: 2m 50s	remaining: 9m 34s
1800:	test: 0.7341630	best: 0.7341630 (1800)	total: 3m 12s	remaining: 9m 15s
2000:	test: 0.7359079	best: 0.7359079 (2000)	total: 3m 34s	remaining: 8m 55s
2200:	test: 0.7375814	best: 0.7375814 (2200)	total: 3m 57s	remaining: 8m 38s
2400:	test: 0.7391823	best: 0.7391823 (2400)	total: 4m 19s	remaining: 8m 16s
2600:	test: 0.

In [54]:
# catfist#
    model2 = CatBoostClassifier(
    iterations=3000,             
    learning_rate=0.03,        
    depth=6,                     
    l2_leaf_reg=3,               
    loss_function="Logloss",   
    eval_metric="AUC",          
    bootstrap_type='Bayesian',   
    random_strength=1.5,        
    bagging_temperature=1.0,     
    random_seed=42,
    od_type="Iter",              
    od_wait=100,                 
    task_type="GPU",       
    devices='0',             
)
model.fit(train_pool , eval_set=valid_pool , verbose=200, use_best_model=True,plot=True)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 197ms	remaining: 9m 50s
200:	total: 38.3s	remaining: 8m 53s
400:	total: 1m 14s	remaining: 8m 2s
600:	total: 1m 23s	remaining: 5m 35s
800:	total: 1m 37s	remaining: 4m 27s
1000:	total: 1m 47s	remaining: 3m 35s
1200:	total: 1m 57s	remaining: 2m 56s
1400:	total: 2m 7s	remaining: 2m 25s
1600:	total: 2m 16s	remaining: 1m 59s
1800:	total: 2m 25s	remaining: 1m 36s
2000:	total: 2m 34s	remaining: 1m 17s
2200:	total: 2m 56s	remaining: 1m 4s
2400:	total: 3m 20s	remaining: 50.1s
2600:	total: 3m 44s	remaining: 34.5s
2800:	total: 4m 9s	remaining: 17.7s
2999:	total: 4m 34s	remaining: 0us


In [65]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

def objective(trial):

    params = {
        "iterations": 3000,
        "depth": trial.suggest_categorical("depth", [4, 5, 6, 7, 8, 9, 10]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.005, 0.01, 0.02, 0.03]),
        "l2_leaf_reg": trial.suggest_categorical("l2_leaf_reg", [1, 3, 5, 7, 10, 15, 20]),
        "bagging_temperature": trial.suggest_categorical("bagging_temperature", [0.1, 0.5, 1.0, 2.0, 5.0]),
        "random_strength": trial.suggest_categorical("random_strength", [0.5, 1, 1.5, 2, 3]),
        "border_count": trial.suggest_categorical("border_count", [32, 64, 128, 254]),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Lossguide"]),

        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "bootstrap_type": "Bayesian",
        "task_type": "GPU",     # GPU tuning
        "devices": "0",
        "random_seed": 42,
    }

    model = CatBoostClassifier(**params)

    model.fit(
        train_pool,
        eval_set=valid_pool,
        use_best_model=True,
        verbose=500
    )

    preds = model.predict_proba(valid_pool)[:, 1]
    auc = roc_auc_score(y_test, preds)

    return auc



study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=100)

print("Best AUC:", study.best_value)
print("Best params:", study.best_params)


[I 2025-12-08 17:56:05,057] A new study created in memory with name: no-name-8b956ce7-b70c-4707-b57a-cd48cb823258
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6856829	best: 0.6856829 (0)	total: 93.6ms	remaining: 4m 40s
500:	test: 0.7041067	best: 0.7041067 (500)	total: 41.5s	remaining: 3m 27s
1000:	test: 0.7093279	best: 0.7093279 (1000)	total: 1m 19s	remaining: 2m 39s
1500:	test: 0.7117762	best: 0.7117762 (1500)	total: 1m 59s	remaining: 1m 59s
2000:	test: 0.7133386	best: 0.7133386 (2000)	total: 2m 39s	remaining: 1m 19s
2500:	test: 0.7140878	best: 0.7140878 (2500)	total: 3m 18s	remaining: 39.5s
2999:	test: 0.7146894	best: 0.7146894 (2999)	total: 3m 57s	remaining: 0us
bestTest = 0.7146894336
bestIteration = 2999


[I 2025-12-08 18:00:12,166] Trial 0 finished with value: 0.7146893908067687 and parameters: {'depth': 8, 'learning_rate': 0.005, 'l2_leaf_reg': 5, 'bagging_temperature': 0.1, 'random_strength': 1, 'border_count': 64, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.7146893908067687.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6716878	best: 0.6716878 (0)	total: 128ms	remaining: 6m 23s
500:	test: 0.7029204	best: 0.7029204 (500)	total: 1m 3s	remaining: 5m 17s
1000:	test: 0.7056317	best: 0.7056317 (1000)	total: 2m 5s	remaining: 4m 10s
1500:	test: 0.7075143	best: 0.7075143 (1500)	total: 3m 7s	remaining: 3m 7s
2000:	test: 0.7085288	best: 0.7085288 (2000)	total: 4m 15s	remaining: 2m 7s
2500:	test: 0.7094353	best: 0.7094381 (2495)	total: 5m 18s	remaining: 1m 3s
2999:	test: 0.7101667	best: 0.7101667 (2999)	total: 6m 20s	remaining: 0us
bestTest = 0.7101666927
bestIteration = 2999


[I 2025-12-08 18:06:39,703] Trial 1 finished with value: 0.710166658249454 and parameters: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 5, 'bagging_temperature': 5.0, 'random_strength': 0.5, 'border_count': 128, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.7146893908067687.
Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.6758702	best: 0.6758702 (0)	total: 83.2ms	remaining: 4m 9s
500:	test: 0.7077560	best: 0.7077560 (500)	total: 40.2s	remaining: 3m 20s
1000:	test: 0.7113009	best: 0.7113009 (1000)	total: 1m 21s	remaining: 2m 43s
1500:	test: 0.7132875	best: 0.7132875 (1500)	total: 2m 3s	remaining: 2m 3s
2000:	test: 0.7146108	best: 0.7146108 (2000)	total: 2m 45s	remaining: 1m 22s
2500:	test: 0.7155896	best: 0.7155896 (2500)	total: 3m 26s	remaining: 41.2s
2999:	test: 0.7162469	best: 0.7162469 (2999)	total: 4m 7s	remaining: 0us
bestTest = 0.7162469029
bestIteration = 2999


[I 2025-12-09 00:48:34,150] Trial 98 finished with value: 0.7162469259765692 and parameters: {'depth': 7, 'learning_rate': 0.03, 'l2_leaf_reg': 1, 'bagging_temperature': 5.0, 'random_strength': 2, 'border_count': 254, 'grow_policy': 'Lossguide'}. Best is trial 86 with value: 0.7280535445434367.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6812755	best: 0.6812755 (0)	total: 71.8ms	remaining: 3m 35s
500:	test: 0.7073006	best: 0.7073006 (500)	total: 35.3s	remaining: 2m 55s
1000:	test: 0.7082405	best: 0.7082405 (1000)	total: 1m 9s	remaining: 2m 19s
1500:	test: 0.7083737	best: 0.7083858 (1355)	total: 1m 44s	remaining: 1m 44s
2000:	test: 0.7082977	best: 0.7083858 (1355)	total: 2m 19s	remaining: 1m 9s
2500:	test: 0.7080342	best: 0.7083858 (1355)	total: 2m 54s	remaining: 34.8s
2999:	test: 0.7078596	best: 0.7083858 (1355)	total: 3m 30s	remaining: 0us
bestTest = 0.7083858252
bestIteration = 1355
Shrink model to first 1356 iterations.


[I 2025-12-09 00:52:11,971] Trial 99 finished with value: 0.7083858318853621 and parameters: {'depth': 5, 'learning_rate': 0.03, 'l2_leaf_reg': 20, 'bagging_temperature': 1.0, 'random_strength': 2, 'border_count': 32, 'grow_policy': 'Lossguide'}. Best is trial 86 with value: 0.7280535445434367.


Best AUC: 0.7280535445434367
Best params: {'depth': 5, 'learning_rate': 0.03, 'l2_leaf_reg': 20, 'bagging_temperature': 1.0, 'random_strength': 2, 'border_count': 254, 'grow_policy': 'Lossguide'}


In [68]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

def objective(trial):

    params = {
        "iterations": 5000,
        "depth": trial.suggest_categorical("depth", [4, 5, 6, 7, 8, 9, 10]),
        "learning_rate": trial.suggest_categorical("learning_rate", [ 0.03,0.05,0.07,0.1]),
        "l2_leaf_reg": trial.suggest_categorical("l2_leaf_reg", [18,19, 20,21,22]),
        "bagging_temperature": trial.suggest_categorical("bagging_temperature", [0.1, 0.5, 1.0, 2.0, 5.0]),
        "random_strength": trial.suggest_categorical("random_strength", [ 2,2.5, 3]),
        "border_count": trial.suggest_categorical("border_count", [ 254,400,300]),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Lossguide"]),
        
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "bootstrap_type": "Bayesian",
        "task_type": "GPU",     # GPU tuning
        "devices": "0",
        "random_seed": 42,
    }

    model = CatBoostClassifier(**params)

    model.fit(
        train_pool,
        eval_set=valid_pool,
        use_best_model=True,
        verbose=500
    )

    preds = model.predict_proba(valid_pool)[:, 1]
    auc = roc_auc_score(y_test, preds)

    return auc



study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=50)

print("Best AUC:", study.best_value)
print("Best params:", study.best_params)


[I 2025-12-09 01:01:30,251] A new study created in memory with name: no-name-6ee0198f-0ed4-4e67-825f-ab7b3d2a5d88
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6812799	best: 0.6812799 (0)	total: 77.3ms	remaining: 6m 26s
500:	test: 0.7225586	best: 0.7225586 (500)	total: 36.1s	remaining: 5m 24s
1000:	test: 0.7254300	best: 0.7254300 (1000)	total: 1m 11s	remaining: 4m 43s
1500:	test: 0.7265065	best: 0.7265065 (1500)	total: 1m 45s	remaining: 4m 7s
2000:	test: 0.7270907	best: 0.7270907 (2000)	total: 2m 22s	remaining: 3m 33s
2500:	test: 0.7274029	best: 0.7274031 (2490)	total: 2m 57s	remaining: 2m 57s
3000:	test: 0.7274821	best: 0.7274938 (2920)	total: 3m 34s	remaining: 2m 22s
3500:	test: 0.7275586	best: 0.7275858 (3465)	total: 4m 9s	remaining: 1m 46s
4000:	test: 0.7274996	best: 0.7275858 (3465)	total: 4m 44s	remaining: 1m 10s
4500:	test: 0.7274638	best: 0.7275858 (3465)	total: 5m 19s	remaining: 35.4s
4999:	test: 0.7273664	best: 0.7275858 (3465)	total: 5m 54s	remaining: 0us
bestTest = 0.7275857627
bestIteration = 3465
Shrink model to first 3466 iterations.


[I 2025-12-09 01:07:34,994] Trial 0 finished with value: 0.7275857846723965 and parameters: {'depth': 5, 'learning_rate': 0.05, 'l2_leaf_reg': 20, 'bagging_temperature': 2.0, 'random_strength': 2, 'border_count': 400, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.7275857846723965.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6858734	best: 0.6858734 (0)	total: 82.6ms	remaining: 6m 52s
500:	test: 0.7229310	best: 0.7229310 (500)	total: 42.9s	remaining: 6m 25s
1000:	test: 0.7254792	best: 0.7254792 (1000)	total: 1m 24s	remaining: 5m 38s
1500:	test: 0.7263115	best: 0.7263115 (1500)	total: 2m 8s	remaining: 4m 59s
2000:	test: 0.7267103	best: 0.7267111 (1980)	total: 2m 51s	remaining: 4m 17s
2500:	test: 0.7270060	best: 0.7270060 (2500)	total: 3m 33s	remaining: 3m 33s
3000:	test: 0.7272232	best: 0.7272232 (3000)	total: 4m 15s	remaining: 2m 50s
3500:	test: 0.7273313	best: 0.7273433 (3345)	total: 4m 57s	remaining: 2m 7s
4000:	test: 0.7273604	best: 0.7273642 (3700)	total: 5m 39s	remaining: 1m 24s
4500:	test: 0.7273365	best: 0.7274120 (4215)	total: 6m 20s	remaining: 42.2s
4999:	test: 0.7273345	best: 0.7274120 (4215)	total: 7m 2s	remaining: 0us
bestTest = 0.727412045
bestIteration = 4215
Shrink model to first 4216 iterations.


[I 2025-12-09 01:14:49,767] Trial 1 finished with value: 0.7274120063277891 and parameters: {'depth': 9, 'learning_rate': 0.03, 'l2_leaf_reg': 19, 'bagging_temperature': 2.0, 'random_strength': 3, 'border_count': 254, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.7275857846723965.
Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.6812799	best: 0.6812799 (0)	total: 79.9ms	remaining: 6m 39s
500:	test: 0.7223573	best: 0.7223573 (500)	total: 37.2s	remaining: 5m 34s
1000:	test: 0.7255553	best: 0.7255553 (1000)	total: 1m 14s	remaining: 4m 55s
1500:	test: 0.7266375	best: 0.7266375 (1500)	total: 1m 51s	remaining: 4m 19s
2000:	test: 0.7271198	best: 0.7271216 (1995)	total: 2m 28s	remaining: 3m 43s
2500:	test: 0.7273127	best: 0.7273194 (2490)	total: 3m 5s	remaining: 3m 5s
3000:	test: 0.7274154	best: 0.7274482 (2910)	total: 3m 41s	remaining: 2m 27s
3500:	test: 0.7274660	best: 0.7274843 (3465)	total: 4m 18s	remaining: 1m 50s
4000:	test: 0.7273842	best: 0.7274843 (3465)	total: 4m 53s	remaining: 1m 13s
4500:	test: 0.7273026	best: 0.7274843 (3465)	total: 5m 29s	remaining: 36.5s
4999:	test: 0.7272326	best: 0.7274843 (3465)	total: 6m 5s	remaining: 0us
bestTest = 0.727484256
bestIteration = 3465
Shrink model to first 3466 iterations.


[I 2025-12-09 07:56:38,487] Trial 48 finished with value: 0.7274842835587032 and parameters: {'depth': 5, 'learning_rate': 0.05, 'l2_leaf_reg': 21, 'bagging_temperature': 2.0, 'random_strength': 3, 'border_count': 254, 'grow_policy': 'Lossguide'}. Best is trial 41 with value: 0.7284182876744737.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6865536	best: 0.6865536 (0)	total: 117ms	remaining: 9m 46s
500:	test: 0.7256024	best: 0.7256024 (500)	total: 45.1s	remaining: 6m 44s
1000:	test: 0.7269871	best: 0.7269871 (1000)	total: 1m 30s	remaining: 6m
1500:	test: 0.7272049	best: 0.7272362 (1375)	total: 2m 15s	remaining: 5m 15s
2000:	test: 0.7271038	best: 0.7272362 (1375)	total: 2m 57s	remaining: 4m 26s
2500:	test: 0.7270551	best: 0.7272362 (1375)	total: 3m 39s	remaining: 3m 39s
3000:	test: 0.7269051	best: 0.7272362 (1375)	total: 4m 22s	remaining: 2m 54s
3500:	test: 0.7266794	best: 0.7272362 (1375)	total: 5m 4s	remaining: 2m 10s
4000:	test: 0.7263532	best: 0.7272362 (1375)	total: 5m 47s	remaining: 1m 26s
4500:	test: 0.7260460	best: 0.7272362 (1375)	total: 6m 30s	remaining: 43.3s
4999:	test: 0.7258221	best: 0.7272362 (1375)	total: 7m 15s	remaining: 0us
bestTest = 0.7272361517
bestIteration = 1375
Shrink model to first 1376 iterations.


[I 2025-12-09 08:04:02,856] Trial 49 finished with value: 0.7272361219688172 and parameters: {'depth': 8, 'learning_rate': 0.05, 'l2_leaf_reg': 21, 'bagging_temperature': 1.0, 'random_strength': 2.5, 'border_count': 300, 'grow_policy': 'Lossguide'}. Best is trial 41 with value: 0.7284182876744737.


Best AUC: 0.7284182876744737
Best params: {'depth': 4, 'learning_rate': 0.05, 'l2_leaf_reg': 21, 'bagging_temperature': 1.0, 'random_strength': 3, 'border_count': 254, 'grow_policy': 'Lossguide'}


In [66]:
modelbestc = CatBoostClassifier(iterations=4000 , learning_rate = 0.05 , grow_policy = 'Lossguide' , 
                                l2_leaf_reg = 21 , depth = 4 ,bagging_temperature=1.0 , random_strength = 3,
                                border_count=254 ,loss_function = 'Logloss',eval_metric='AUC',bootstrap_type = 'Bayesian',
                                task_type='GPU',devices='0',random_seed=42)

In [77]:
modelbestc.fit(all_train_Pool ,verbose = 200)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 62.6ms	remaining: 3m 39s
200:	total: 11.1s	remaining: 3m 2s
400:	total: 22.1s	remaining: 2m 50s
600:	total: 33s	remaining: 2m 39s
800:	total: 44s	remaining: 2m 28s
1000:	total: 55s	remaining: 2m 17s
1200:	total: 1m 5s	remaining: 2m 6s
1400:	total: 1m 16s	remaining: 1m 54s
1600:	total: 1m 27s	remaining: 1m 43s
1800:	total: 1m 37s	remaining: 1m 32s
2000:	total: 1m 48s	remaining: 1m 21s
2200:	total: 1m 59s	remaining: 1m 10s
2400:	total: 2m 10s	remaining: 59.7s
2600:	total: 2m 21s	remaining: 48.8s
2800:	total: 2m 32s	remaining: 38s
3000:	total: 2m 43s	remaining: 27.1s
3200:	total: 2m 54s	remaining: 16.3s
3400:	total: 3m 4s	remaining: 5.38s
3499:	total: 3m 10s	remaining: 0us


In [81]:
pred_modelbestc = modelbestc.predict_proba(test)[:,1]

In [82]:
submission = pd.DataFrame({
    "id": test.index,
    "diagnosed_diabetes": pred_modelbestc
})

In [84]:
submission.to_csv('uptuna.csv',index=False)

In [86]:
pred_modelbestc

array([0.48105482, 0.67756586, 0.76483147, ..., 0.67203563, 0.56505177,
       0.61278214], shape=(300000,))

In [14]:
import xgboost as xgb
import optuna

In [16]:
def objective(trial):

    params = {
        "n_estimators": 4000,

        "max_depth": trial.suggest_categorical("max_depth", [3, 4, 5, 6, 7, 8, 9, 10]),
        "min_child_weight": trial.suggest_categorical("min_child_weight", [1, 2, 3, 4, 5, 6, 8, 10]),
        "gamma": trial.suggest_categorical("gamma", [0, 0.1, 0.2, 0.5, 1, 2, 5]),

        "reg_alpha": trial.suggest_categorical("reg_alpha", [0, 0.1, 0.5, 1, 3, 5, 10]),
        "reg_lambda": trial.suggest_categorical("reg_lambda", [0.1, 0.5, 1, 2, 5, 10, 20]),

        "learning_rate": trial.suggest_categorical("learning_rate", [0.005, 0.01, 0.02, 0.03, 0.08, 0.1]),

        "subsample": trial.suggest_categorical("subsample", [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        "colsample_bytree": trial.suggest_categorical("colsample_bytree", [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),

        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),

        "booster": "gbtree",
        "tree_method": "gpu_hist",
        "predictor": "gpu_predictor",

        "objective": "binary:logistic",
        "eval_metric": "auc",

        "random_state": 42,
        "enable_categorical":True
    }

    modelX = xgb.XGBClassifier(**params)

    modelX.fit(X_train , y_train , verbose=500  , eval_set=[(X_test,y_test)])

    proba = modelX.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, proba)

    return auc     
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best AUC:", study.best_value)
print("Best params:", study.best_params)



[I 2025-12-09 12:55:47,946] A new study created in memory with name: no-name-c4376260-ac9d-44ff-a833-f6c484e968f5


[0]	validation_0-auc:0.67274
[500]	validation_0-auc:0.72518
[1000]	validation_0-auc:0.72644
[1500]	validation_0-auc:0.72645
[2000]	validation_0-auc:0.72618
[2500]	validation_0-auc:0.72571
[3000]	validation_0-auc:0.72516
[3500]	validation_0-auc:0.72467
[3999]	validation_0-auc:0.72407


[I 2025-12-09 12:56:50,875] Trial 0 finished with value: 0.7240714220926033 and parameters: {'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.5, 'reg_alpha': 3, 'reg_lambda': 10, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.6, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.7240714220926033.


[0]	validation_0-auc:0.68208
[500]	validation_0-auc:0.72514
[1000]	validation_0-auc:0.72596
[1500]	validation_0-auc:0.72541
[2000]	validation_0-auc:0.72452
[2500]	validation_0-auc:0.72357
[3000]	validation_0-auc:0.72266
[3500]	validation_0-auc:0.72156
[3999]	validation_0-auc:0.72063


[I 2025-12-09 12:58:22,916] Trial 1 finished with value: 0.7206267042257685 and parameters: {'max_depth': 5, 'min_child_weight': 1, 'gamma': 0, 'reg_alpha': 3, 'reg_lambda': 5, 'learning_rate': 0.08, 'subsample': 0.7, 'colsample_bytree': 0.8, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.7240714220926033.


...

[0]	validation_0-auc:0.62522
[500]	validation_0-auc:0.72258
[1000]	validation_0-auc:0.72549
[1500]	validation_0-auc:0.72620
[2000]	validation_0-auc:0.72646
[2500]	validation_0-auc:0.72662
[3000]	validation_0-auc:0.72672
[3500]	validation_0-auc:0.72678
[3999]	validation_0-auc:0.72682


[I 2025-12-09 16:28:39,903] Trial 98 finished with value: 0.7268172443599986 and parameters: {'max_depth': 6, 'min_child_weight': 3, 'gamma': 2, 'reg_alpha': 0, 'reg_lambda': 20, 'learning_rate': 0.03, 'subsample': 0.9, 'colsample_bytree': 0.5, 'grow_policy': 'lossguide'}. Best is trial 32 with value: 0.7272557764369656.


[0]	validation_0-auc:0.68127
[500]	validation_0-auc:0.71072
[1000]	validation_0-auc:0.71657
[1500]	validation_0-auc:0.72001
[2000]	validation_0-auc:0.72231
[2500]	validation_0-auc:0.72367
[3000]	validation_0-auc:0.72452
[3500]	validation_0-auc:0.72521
[3999]	validation_0-auc:0.72576


[I 2025-12-09 16:30:10,885] Trial 99 finished with value: 0.7257643378935 and parameters: {'max_depth': 5, 'min_child_weight': 3, 'gamma': 1, 'reg_alpha': 5, 'reg_lambda': 10, 'learning_rate': 0.01, 'subsample': 0.9, 'colsample_bytree': 0.7, 'grow_policy': 'lossguide'}. Best is trial 32 with value: 0.7272557764369656.


Best AUC: 0.7272557764369656
Best params: {'max_depth': 5, 'min_child_weight': 2, 'gamma': 0, 'reg_alpha': 5, 'reg_lambda': 10, 'learning_rate': 0.02, 'subsample': 0.9, 'colsample_bytree': 0.5, 'grow_policy': 'lossguide'}


In [1]:
import numpy as np
import xgboost as xgb

print("NumPy :", np.__version__)
print("XGBoost :", xgb.__version__)
print("XGBoost path:", xgb.__file__)


NumPy : 1.26.4
XGBoost : 1.7.6
XGBoost path: C:\Users\youne\anaconda3\envs\torch_gpu\lib\site-packages\xgboost\__init__.py


In [1]:
import xgboost as xgb
print(xgb.__version__)
print(xgb)



1.7.6
<module 'xgboost' from 'C:\\Users\\youne\\anaconda3\\envs\\torch_gpu\\lib\\site-packages\\xgboost\\__init__.py'>


In [19]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [37]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from catboost import CatBoostClassifier

st_c = StackingClassifier(
    estimators=[
        ('cat', CatBoostClassifier(
            cat_features=cat_features,
            iterations=3500,
            learning_rate=0.05,
            grow_policy='Lossguide',
            l2_leaf_reg=21,
            depth=4,
            bagging_temperature=1.0,
            random_strength=3,
            border_count=254,
            loss_function='Logloss',
            eval_metric='AUC',
            bootstrap_type='Bayesian',   
            task_type='GPU',
            devices='0',                
            random_seed=42,
            verbose=False               
        )),

        ('xgb', xgb.XGBClassifier(
            n_estimators = 5000,
            max_depth=5,
            min_child_weight=2,
            gamma=0,
            reg_alpha=5,
            reg_lambda=10,
            learning_rate=0.02,
            subsample=0.9,
            colsample_bytree=0.5,
            grow_policy='lossguide',
            booster='gbtree',
            tree_method='gpu_hist',      
            predictor='gpu_predictor',
            enable_categorical=True,     
            objective='binary:logistic',
            eval_metric='auc',
            random_state=42
        ))
    ],

    final_estimator=LogisticRegression(max_iter=5000),
    cv=5
)


In [38]:
st_c.fit(X,y)

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


,estimators,"[('cat', ...), ('xgb', ...)]"
,final_estimator,LogisticRegre...max_iter=5000)
,cv,5
,stack_method,'auto'
,n_jobs,None
,passthrough,False
,verbose,0
,objective,'binary:logistic'
,use_label_encoder,None
,base_score,None
,booster,'gbtree'


In [40]:
pred_stack = st_c.predict_proba(test)[:,1]

In [41]:
submission = pd.DataFrame({
    "id": test.index,
    "diagnosed_diabetes": pred_stack
})

In [42]:
submission.to_csv('stack.csv',index=False)

In [76]:
X_feature_train = X_train.copy()

X_feature_train['bmi*age'] = X_train['bmi']*X_train['age']
X_feature_train.columns

Index(['age', 'alcohol_consumption_per_week',
       'physical_activity_minutes_per_week', 'diet_score',
       'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi',
       'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'education_level',
       'income_level', 'smoking_status', 'employment_status',
       'family_history_diabetes', 'hypertension_history',
       'cardiovascular_history', 'bmi*age'],
      dtype='object')

In [77]:
X_feature_test = X_test.copy()

X_feature_test['bmi*age'] = X_test['bmi']*X_test['age']

In [78]:
pool_feature_train= Pool(X_feature_train,y_train,cat_features = cat_features )
pool_feature_valid = Pool(X_feature_test , y_test , cat_features = cat_features)

In [79]:
modelbestc.fit( pool_feature_train, eval_set=pool_feature_valid , verbose=200, use_best_model=True)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6790362	best: 0.6790362 (0)	total: 42.3ms	remaining: 2m 49s
200:	test: 0.7122515	best: 0.7122515 (200)	total: 6.22s	remaining: 1m 57s
400:	test: 0.7197440	best: 0.7197440 (400)	total: 12s	remaining: 1m 47s
600:	test: 0.7226217	best: 0.7226217 (600)	total: 18.3s	remaining: 1m 43s
800:	test: 0.7243241	best: 0.7243241 (800)	total: 24s	remaining: 1m 35s
1000:	test: 0.7254959	best: 0.7254959 (1000)	total: 29.7s	remaining: 1m 28s
1200:	test: 0.7261461	best: 0.7261461 (1200)	total: 35.4s	remaining: 1m 22s
1400:	test: 0.7266753	best: 0.7266783 (1395)	total: 40.8s	remaining: 1m 15s
1600:	test: 0.7270358	best: 0.7270358 (1600)	total: 46.5s	remaining: 1m 9s
1800:	test: 0.7272727	best: 0.7272786 (1795)	total: 52.1s	remaining: 1m 3s
2000:	test: 0.7274577	best: 0.7274577 (2000)	total: 57.8s	remaining: 57.7s
2200:	test: 0.7276077	best: 0.7276077 (2200)	total: 1m 3s	remaining: 51.9s
2400:	test: 0.7277091	best: 0.7277091 (2400)	total: 1m 8s	remaining: 45.9s
2600:	test: 0.7277994	best: 0.7278